## Parametros

In [ ]:
LKH_MDE = "LKH_MDE"

## Cálculos en Gold
Leer de la tabla h_transactions de Silver, hacer calculos y escribir amount_by_merchantCategory en Gold

In [ ]:
from pyspark.sql.functions import round, col

# Leer los datos de la tabla 'transactions' desde el esquema 
df = spark.read.table(f"{LKH_MDE}.silver.h_transactions")

# Convertir el campo 'amount' a tipo numérico (double)
df = df.withColumn("amount", df["amount"].cast("double"))

# Calcular el importe total por merchantCategory
total_amount_by_market = df.groupBy("merchantCategory").sum("amount").withColumnRenamed("sum(amount)", "total_amount")

# Redondear la columna 'total_amount' 
total_amount_by_market = total_amount_by_market.withColumn("total_amount", round(col("total_amount"), 2))

# Crear el esquema 'gold' si no existe 
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

# Guardar el resultado en la capa 'gold'
total_amount_by_market.write.mode("overwrite").saveAsTable("gold.amount_by_merchantCategory_v1")

# Mostrar el resultado
total_amount_by_market.show()